<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Caso-de-estudio-1.2.2:-Agrupamiento-espectral:-Agrupación-de-noticias" data-toc-modified-id="Caso-de-estudio-1.2.2:-Agrupamiento-espectral:-Agrupación-de-noticias-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Caso de estudio 1.2.2: Agrupamiento espectral: Agrupación de noticias</a></span></li><li><span><a href="#Generación-de-la-base-de-datos-(Web-Scraping)" data-toc-modified-id="Generación-de-la-base-de-datos-(Web-Scraping)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generación de la base de datos (Web Scraping)</a></span></li><li><span><a href="#Importación-de-la-base-de-datos" data-toc-modified-id="Importación-de-la-base-de-datos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importación de la base de datos</a></span></li><li><span><a href="#Generación-de-atributos" data-toc-modified-id="Generación-de-atributos-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Generación de atributos</a></span></li></ul></div>

# Caso de estudio 1.2.2: Agrupamiento espectral: Agrupación de noticias

---
<br>

Este caso de estudio considera una base de datos de artículos de prensa, sobre diferentes temas, y usa _clusterización espectral_ para agruparlos dependiendo de la frecuencia de ciertas palabras. Este notebook proporciona el código para generar la base de datos, pero también puede enocontrar un ejemplo de base de datos en la carpeta `/Data`. Esta base de datos se generó el día 29 de mayo de 2020 mediante técnicas de minería de datos (_web scraping_).

Este caso de estudio usa la librería [`mitie`](https://github.com/mit-nlp/MITIE), desarrollada en MIT. Todos los pasos para instalar tnato la librería como el modelo NER usado en este caso de estudio pueden encontrarse en la documentación online.

<br>

---

Configuración del notebook:

In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import csv

#ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cluster

#Web scraping libraries
from bs4 import BeautifulSoup

#NLP libraries
from mitie import *
print("loading NER model...")
ner = named_entity_extractor('/Users/inigo/opt/anaconda3/lib/python3.7/site-packages/mitie/models/MITIE-models/english/ner_model.dat')
print("\nTags output by this NER model:", ner.get_possible_ner_tags())

loading NER model...

Tags output by this NER model: ['PERSON', 'LOCATION', 'ORGANIZATION', 'MISC']


# Generación de la base de datos (Web Scraping)

En este ejemplo, artículos de 8 temas diferentes del periódico __The Guardian__ han sido recopilados. Los pasos a seguir para crear la base de datos son:

1. Obtener el código fuente de la web principal de The Guardian, y almacenar los links a las secciones (temas) de interés.
2. Iterar la lista de links y obtener la información de 10 artículos por sección (título y contenido).
3. Guardar los artículos, títulos, y temas en archivos `.txt`.

In [2]:
UK_news_url = 'https://www.theguardian.com/uk'
#Descargando los links de los diferentes temas
html_data = requests.get(UK_news_url).text
soup = BeautifulSoup(html_data, 'html.parser')
url_topics = [el.find('a')['href'] for el in soup.find_all(class_ = 'subnav__item')[1:9]]
topics = [el.text.strip('\n').replace(' ','_') for el in soup.find_all(class_ = 'subnav-link')[1:9]]
for i in range(len(topics)):
    print('Topic {}: {} ({})'.format(i+1,topics[i],url_topics[i]))


Topic 1: Elections_2020 (https://www.theguardian.com/us-news/us-elections-2020)
Topic 2: World (https://www.theguardian.com/world)
Topic 3: Environment (https://www.theguardian.com/us/environment)
Topic 4: Soccer (https://www.theguardian.com/football)
Topic 5: US_Politics (https://www.theguardian.com/us-news/us-politics)
Topic 6: Business (https://www.theguardian.com/us/business)
Topic 7: Tech (https://www.theguardian.com/us/technology)
Topic 8: Science (https://www.theguardian.com/science)


In [3]:
def save_to_txt(filename, content):
    '''
    Creates a new .txt file with as specific name in the Data directory
    '''
    with open(r"Data/{}.txt".format(filename), "w") as f:
        print(content, file=f)

article_titles = []
article_contents = []
article_topics = []
articles_per_topic = 10
n = 1
for topic, url_topic in list(zip(topics,url_topics)):
    #Getting the first 15
    soup = BeautifulSoup(requests.get(url_topic).text, 'html.parser')
    url_articles = [el.find('a')['href'] for el in soup.find_all(class_ = 'fc-item__content')]
    print('\n{}:'.format(topic))
    i = 0
    while article_topics.count(topic) < articles_per_topic:
        soup = BeautifulSoup(requests.get(url_articles[i]).text, 'html.parser')
        try:
            title = soup.find(class_ = 'content__headline').text.strip('\n')
            content = ' '.join([el.text for el in soup.find(class_ = 'content__article-body from-content-api js-article__body').find_all('p')])
            i += 1
            if i == len(url_articles):
                print('Only {} articles found in \"{}"'.format(article_topics.count(topic),topic))
                break
            if title not in article_titles:
                article_titles += [title]
                article_contents += [content]
                article_topics += [topic]
                save_to_txt('title-{}'.format(n),title)
                save_to_txt('article-{}'.format(n),content)
                save_to_txt('topic-{}'.format(n),topic)
                print('{}'.format(title))
                n += 1
                if round(len(article_titles)/10) == len(article_titles)/10:
                    print('Article count: {}'.format(len(article_titles)))
        except:
            i += 1
            if i == len(url_articles):
                print('Only {} articles found in \"{}"'.format(article_topics.count(topic),topic))
                break
            pass
        
                
df = pd.DataFrame({'topic':article_topics,'title':article_titles,'content':article_contents})


Elections_2020:
Biden sets solemn tone as Trump waits 15 hours to mark Covid-19 milestone
Revealed: conservative group fighting to restrict voting tied to powerful dark money network
Twitter labels Trump's false claims with warning for first time
'Feels good to be out of my house': Biden lays Memorial Day wreath in Delaware
Why is Trump so restrained about the Biden sexual assault allegation?
Swing states become partisan battlegrounds in America's fight against Covid-19
Trump campaign focuses fire on Biden as pandemic undermines strategy
'Obamagate': Fox News focuses on conspiracy theory rather than Covid-19
‘The United States is broken as hell’ – the division in politics over race and class
Socialism used to be a dirty word. Is America now ready to embrace it?
Article count: 10

World:
Global report: South Korea postpones school reopening due to new outbreak
'Gross incompentence at highest levels': ex-Obama adviser blasts Trump's Covid response
Monkeys steal Covid-19 test samples fro

# Importación de la base de datos

Una vez tenemos la base de datos guardada en carpeta deseada, podemos usar el código del caso de estudio para importar la información.

In [4]:
#número total de artículos para procesar.
N = 80
#para almacenar los temas, títulos y contenidos de las noticias:
topics_array = []
titles_array = []
corpus = []
for i in range(1, N+1):
    #consiga el contenido del artículo.
    with open('Data/article-' + str(i) + '.txt', 'r') as myfile:
        d1=myfile.read().replace('\n', '')
        d1 = d1.lower()
        corpus.append(d1)
    #consiga el tema original del artículo.
    with open('Data/topic-' + str(i) + '.txt', 'r') as myfile:
        to1=myfile.read().replace('\n', '')
        to1 = to1.lower()
        topics_array.append(to1)
    #consiga el título del artículo.
    with open('Data/title-' + str(i) + '.txt', 'r') as myfile:
        ti1=myfile.read().replace('\n', '')
        ti1 = ti1.lower()
        titles_array.append(ti1)

# Generación de atributos

Para generar los atributos de cada instancia (artículo):

1. Enlazamos todos los corpus de texto de artículos para determinar todas las palabras únicas que se utilizan en el conjunto de datos.
2. Buscamos el subconjunto de las entidades del modelo NER que se encuentra entre las palabras únicas que se utilizan en el conjunto de datos (determinado en el paso 1).

In [5]:
#vector de subconjunto de entidades
entity_text_array = [] 
for i in range(1, N+1):
    #cargue el archivo de texto con el contenido del artículo y conviértalo en una lista de palabras
    tokens = tokenize(load_entire_file(('Data/article-' + str(i) + '.txt')))
    #extraiga todas las entidades conocidas del modelo ner mencionado en este artículo
    entities = ner.extract_entities(tokens)
    #extraiga las palabras de entidades reales y agréguelas al vector
    for e in entities: 
        range_array = e[0]
        tag = e[1]
        score = e[2]
        score_text = "{:0.3f}".format(score)
        entity_text = " ".join(str(tokens[j]) for j in range_array) 
        entity_text_array.append(entity_text.lower())
#elimine las entidades duplicadas que se hayan detectado
#entity_text_array = np.unique(entity_text_array)
entity_text_array = list(set(entity_text_array))

Ahora que ya tenemos la lista de todas las entidades utilizadas en la base de datos, podemos representar cada artículo como un vector que contiene la puntuación de [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) para cada entidad almacenada en el `entity_text_array`. Esta tarea se puede realizar fácilmente con la librería [scikit-learn](http://scikit- learn.org/stable/) de Python

In [6]:
vect = TfidfVectorizer(sublinear_tf=True, max_df=0.5, analyzer='word',
                       stop_words='english', vocabulary=entity_text_array)
corpus_tf_idf = vect.fit_transform(corpus)


Ahora que tenemos los artículos representados por sus atributos (puntuaciones de TF-IDF), podemos llevar a cabo el agrupamiento espectral de los mismos usando la librería `scikit-learn` de nuevo

In [7]:
# change n_clusters to equal the number of clusters desired
n_clusters = 8
#spectral clustering
spectral = cluster.SpectralClustering(n_clusters= n_clusters, 
                                      eigen_solver='arpack', 
                                      affinity="nearest_neighbors", 
                                      n_neighbors = 10)
spectral.fit(corpus_tf_idf)

SpectralClustering(affinity='nearest_neighbors', assign_labels='kmeans',
                   coef0=1, degree=3, eigen_solver='arpack', eigen_tol=0.0,
                   gamma=1.0, kernel_params=None, n_clusters=8,
                   n_components=None, n_init=10, n_jobs=None, n_neighbors=10,
                   random_state=None)

Finalmente las siguientes líneas de código permiten ver el output en el siguiente formato (una línea por artículo):

<br>

<center>__no. artículo, tema, cluster, título__</center>

In [8]:
if hasattr(spectral, 'labels_'):
    cluster_assignments = spectral.labels_.astype(np.int)
    for i in range(0, len(cluster_assignments)):
        print (i, topics_array[i], cluster_assignments [i], titles_array[i])

0 elections_2020 7 biden sets solemn tone as trump waits 15 hours to mark covid-19 milestone
1 elections_2020 1 revealed: conservative group fighting to restrict voting tied to powerful dark money network
2 elections_2020 7 twitter labels trump's false claims with warning for first time
3 elections_2020 0 'feels good to be out of my house': biden lays memorial day wreath in delaware
4 elections_2020 0 why is trump so restrained about the biden sexual assault allegation?
5 elections_2020 1 swing states become partisan battlegrounds in america's fight against covid-19
6 elections_2020 4 trump campaign focuses fire on biden as pandemic undermines strategy
7 elections_2020 0 'obamagate': fox news focuses on conspiracy theory rather than covid-19
8 elections_2020 7 ‘the united states is broken as hell’ – the division in politics over race and class
9 elections_2020 7 socialism used to be a dirty word. is america now ready to embrace it?
10 world 0 global report: south korea postpones school

Como puede observarse, el algoritmo no clasifica los artículos según las secciones de las que se han obtenido. Puede indagar más a fondo en los parámetros del modelo para mejorar dichos resultados, o buscar una explicación para entender el criterio con el que el algoritmo está agrupando los artículos.

<br>

<br>

<br>

<br>

<br>

<br>

Created by Iñigo de la Maza. Contact: [idelamaza.com](https://idelamaza.github.io/)

<br>

<br>

<br>

In [5]:
## IGNORE THE CODE BELOW ##

#Getting names of imported libraries and versions for creating a requirements.txt file
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
            
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
requirements.append(('beautifulsoup4', '4.9.1'))    
#Getting the packages already included in requirements.txt
with open(r"../../requirements.txt", "r") as f:
    pkgs = [pkg.split('==')[0] for pkg in f.readlines()]
#Adding missing packages
print('List of packages and versions:\n')     
with open(r"../../requirements.txt", "a") as f:
    for r in requirements:
        print("{}=={}".format(*r))
        if r[0] not in pkgs:
            f.write("{}=={}\n".format(*r))

List of packages and versions:

scikit-learn==0.22.1
requests==2.23.0
pandas==1.0.0
numpy==1.18.1
mitie==0.7.36
matplotlib==3.1.3
beautifulsoup4==4.9.1
